In [1]:
import sys
sys.path.append('/home/simon/git/lda/code/')

In [2]:
from ms2lda_feature_extraction import LoadMGF

In [3]:
mgf_file = '/home/simon/Dropbox/MolecularNetworking_GCF-MF_Crusemann_Duncan/Crusemann_only_Clutered_Data/METABOLOMICS-SNETS-c36f90ba-download_clustered_spectra-main.mgf'

In [4]:
min_ms2_intensity = 25

In [5]:
l = LoadMGF(min_ms2_intensity = 25)
ms1,ms2,metadata = l.load_spectra([mgf_file])

Filtering MS2 on intensity
85977 MS2 remaining


In [6]:
%load_ext autoreload
%autoreload 2

In [56]:
mols_as_dicts = {}
for m in ms1:
    mols_as_dicts[m.name] = []

In [57]:
for f in ms2:
    m = f[3]
    mols_as_dicts[m.name].append((f[0],f[2]))

In [58]:
for m in mols_as_dicts:
    mols_as_dicts[m] = sorted(mols_as_dicts[m],key = lambda x: x[0])

In [59]:
for m in mols_as_dicts:
    max_val = max([x[1] for x in mols_as_dicts[m]])
    new_spec = []
    for mz,i in mols_as_dicts[m]:
        new_spec.append((mz,100.0*i/max_val))
    mols_as_dicts[m] = new_spec

In [60]:
parentmass = {}
for m in ms1:
    parentmass[m.name] = metadata[m.name].get('parentmass')

Compute all the scores

In [61]:
scores = []
n_mols = len(ms1)
import numpy as np


In [ ]:
from spectrum_alignment import score_alignment
tol = 0.02
n_done = 0
max_align = 0
max_score = 0
for i,m1 in enumerate(ms1[:-1]):
    for j,m2 in enumerate(ms1[i+1:]):
        score,alignment = score_alignment(mols_as_dicts[m1.name],mols_as_dicts[m2.name],parentmass[m1.name],parentmass[m2.name],tol)
#         scoremat[i,j] = score
#         n_matches[i,j] = len(alignment)
        if len(alignment) > 0:
            scores.append((m1.name,m2.name,score,len(alignment)))
            if len(alignment) > max_align:
                max_align = len(alignment)
#                 print max_align
            if score > max_score:
                max_score = score
#                 print max_score
    n_done += 1
    if n_done % 100 == 0:
        print n_done,n_mols
#         scores.append((m1,m2,score))
    


100 5930
200 5930
300 5930
400 5930


In [64]:
from mol_networks import create_edge_dict,Network
min_frag_overlap = 6
min_score = 0.5
mnax_component_size = 100
edge_dict = create_edge_dict(scores,min_frag_overlap = min_frag_overlap,min_score = min_score)
network = Network(edge_dict)
man_network = network.copy()
man_network.top_k_filter(k=10)
components = man_network.convert_to_components(max_component_size=max_component_size,min_score = min_score)

('document_1', 'document_2', 0.0998928382327748, 1)


In [74]:
print "There are {} components".format(len(components))
print
print
for component in components:
    print
    print component

There are 23 components



Network has 7 nodes
document_252 (degree = 2)	(document_251,0.659064319634) (document_211,0.685267732658)
document_251 (degree = 6)	(document_252,0.659064319634) (document_254,0.888682078758) (document_211,0.938789185647) (document_7,0.587784175877) (document_413,0.777971564404) (document_20,0.55787402163)
document_254 (degree = 1)	(document_251,0.888682078758)
document_211 (degree = 2)	(document_252,0.685267732658) (document_251,0.938789185647)
document_7 (degree = 2)	(document_251,0.587784175877) (document_20,0.864137009229)
document_413 (degree = 1)	(document_251,0.777971564404)
document_20 (degree = 2)	(document_251,0.55787402163) (document_7,0.864137009229)


Network has 39 nodes
document_257 (degree = 2)	(document_262,0.819038374997) (document_261,0.842642128675)
document_256 (degree = 3)	(document_264,0.812746405727) (document_272,0.716736830346) (document_261,0.903914418288)
document_397 (degree = 2)	(document_292,0.650436116549) (document_404,0.66485

In [75]:
import csv
with open('crusemann_edges.csv','w') as f:
    writer = csv.writer(f)
    network.write_network(writer,heads = True)

In [76]:
with open('crusemann_components.csv','w') as f:
    writer = csv.writer(f)
    for i,component in enumerate(components):
        if i==0:
            heads = True
        else:
            heads = False
        component.write_network(writer,heads = heads)